In [1]:
import pandas as pd
import numpy as np
from datetime import date, time
from IPython.display import display
pd.options.display.max_columns = None
import os
import re

## Functions

In [3]:
#function for creating lists of SIDs
def listify(cell):
    if pd.isna(cell):
        return []
    else:
        cell = str(cell)
        cell = cell.replace(" ", "")
        listy = list(cell.split(","))
        return listy

In [839]:
#function for correcting double counts for same concept
def no_double(component, *args):
    component = pd.DataFrame(component)
    count = 1
    for data in args:
        if not isinstance(data, pd.Series):
            for column in data:
                name = 'arg' + str(count)
                column_series = pd.Series(column)
                component[name] = column_series
                count += 1
        else:
            component = pd.concat((component, data), axis=1)
    
    component['sum'] = component.iloc[:, 1:].sum(axis=1)

    updated_component = []
    
    for row in component.iterrows():
        if row[1][0] >0 and row[1][-1] >0:
            updated_component.append(0)
        elif row[1][0] == 1 and row[1][-1] == 0:
            updated_component.append(1)
        else:
            updated_component.append(0)
    
    return updated_component

In [415]:
#function for converting operative severity to numbers
def op_severity(severity, SID):
    severity_series = severity + SID
    updated_severity = []
    
    for item in severity_series:
        item = str(item)
        item = item.lower()
        if 'complex' in item:
            updated_severity.append(5)
        elif 'xmajor' in item:
            updated_severity.append(4)
        elif 'major' in item:
            updated_severity.append(3)
        elif 'intermediate' in item:
            updated_severity.append(2)
        elif 'minor' in item:
            updated_severity.append(1)
        else:
            updated_severity.append(3)
            print(item)
            
    return updated_severity

In [253]:
#function for calculating CCI
def cci(row):
    comorbidities = row.loc[['cardiac', 'kidney', 'liver', 'neuro', 'pulm', 'dementia', 'diabetes', 'comp_diabetes', 'malignancy', 'metastasis', 'pvd', 'rheum']]
    
    #adjust weights
    comorbidities['kidney'] = comorbidities['kidney'] * 2
    comorbidities['liver'] = comorbidities['liver'] * 2
    comorbidities['neuro'] = comorbidities['neuro'] * 2
    comorbidities['comp_diabetes'] = comorbidities['comp_diabetes'] * 2
    comorbidities['malignancy'] = comorbidities['malignancy'] * 2
    comorbidities['metastasis'] = comorbidities['metastasis'] * 6
    
    #calculate score
    score = comorbidities.sum()
    return score

In [636]:
#functions for identifying post-op crit care for patients admitted to level 0 & 1
def crit_care(p):
    updated_crit_care = []
    for row in p.iterrows():
        if row[1][30] == 1:
            updated_crit_care.append(0)
        elif row[1][57] > 1:
            updated_crit_care.append('Not applicable')
        elif (row[1][37] + row[1][37]) >0:
            updated_crit_care.append(1)
        else:
            updated_crit_care.append(0)
    return updated_crit_care

def crit_care_7d(p, ts):
    updated_crit_care = []
    for row in p.iterrows():
        if row[1][30] == 1: #cancelled
            updated_crit_care.append(0)
        elif row[1][57] > 1: #level 2-3 post-op
            updated_crit_care.append('Not applicable')
        elif (row[1][37] + row[1][37]) >0: #LOS in levels 2-3 >0
            SID = row[0]
            date = row[1][29]
            date = date.replace(hour=0, minute=0)
            date_position = ts.index.get_loc(date)
            date_7 = date_position + 7
            check = ts.iloc[date_position:date_7, [25, 29]]
            check = check['adm_7d_lvl2'] + check['adm_7d_lvl3']
            count = 0
            for row in check:
                if SID in row:
                    updated_crit_care.append(1)
                    count += 1
                    break
            if count == 0:
                updated_crit_care.append(0)
        else:
            updated_crit_care.append(0)
    return updated_crit_care

In [743]:
#function for adding dates of death from encounter end
def died_date(died, date, encounter_end):
    checker = pd.concat((died, date, encounter_end), axis=1)
    updated_date = []
    
    for row in checker.iterrows():
        if row[1]['died'] == 0:
            updated_date.append(0)
        if row[1]['died'] == 1:
            if row[1]['died_date'] == 0:
                updated_date.append(row[1]['encounter_end'])
        if row[1]['died'] == 1:
            if row[1]['died_date'] != 0:
                updated_date.append(row[1]['died_date'])
    return updated_date

In [855]:
#function for loading data for each hospital
def data_load(p, ts):

    #loading patient_crf
    patient = pd.read_excel("C:\\Users\\chris\\Documents\\Job Documents\\Portfolio Evidence\\Research\\Post-Operative Critical Care Beds Project\\Results Data\\Cleaned CRFs\\"+p, sheet_name=None)
    try:
        patient = pd.DataFrame(data=patient['Sheet1'])
    except:
        print('The sheet isnt called Sheet1, rename this')
        
    patient.columns = patient.iloc[2]
    patient = patient.iloc[3:]
    patient.columns.name = None
    
    patient = patient.iloc[:len(patient['sid_number'].explode().dropna())]
    
    if patient.columns[0] == 'hospital_number':
        patient = patient.iloc[:, 1:]
        
    patient.fillna(value=0, inplace=True)
    
    patient['diabetes'] = no_double(patient['diabetes'], patient['comp_diabetes'])
    patient['malignancy'] = no_double(patient['malignancy'], patient['metastasis'])
    patient['clin_j'] = no_double(patient['clin_j'], (patient['sop'], patient['cpet'], patient['risk_score']))
    patient['op_severity'] = op_severity(patient['op_severity'], patient['sid_number'])
    patient['died_date'] = died_date(patient['died'], patient['died_date'], patient['encounter_end'])
    
    patient['risk_score'] = [1 if (row[1]['mortality_score'] + row[1]['morbidity_score']) > 0 else 0 for row in patient.iterrows()]
    
    if 'scheduled_date' in patient.columns:
        merged = []
        for row in patient.iterrows():
            row_data = row[1]
            time_merge = pd.Timestamp.combine(row_data['scheduled_date'], row_data['scheduled_start'])
            merged.append(time_merge)
        merged_series = pd.DataFrame(merged)
        merged_series = merged_series.set_index(patient['sid_number'])
        patient['scheduled_start'] = merged_series
        patient = patient.drop('scheduled_date', axis=1)
        
    if 'rhem' in patient.columns:
        patient = patient.rename(columns={'rhem': 'rheum'})
        
    patient = patient.set_index('sid_number')
    patient.index.name = None
        
    code_num = str(patient.index[0])
    code = code_num[:5]

    if len(patient.columns) != 62:
        print("The Patient CRF for site "+code+" is the incorrect shape")
        patient.info()
    if list(patient.columns) != ['age', 'gender_male', 'gender_female', 'gender_non_binary', 'asian', 'black', 'mixed', 'white', 'other', 'cardiac', 'kidney', 'liver', 'neuro', 'pulm', 'dementia', 'diabetes', 'comp_diabetes', 'htn', 'malignancy', 'metastasis', 'pvd', 'rheum', 'surgical_speciality', 'proposed_procedure', 'cancer_surgery', 'ncepod_class', 'op_severity', 'scheduled_start', 'previous_cancellation', 'kts', 'cancelled', 'critical_care_7_days', 'encounter_start', 'encounter_end', 'los_encounter', 'los_0', 'los_1', 'los_2', 'los_3', 'died', 'died_date', 'cancel_nobed', 'cancel_other', 'cancel_other_text', 'sop', 'cpet', 'clin_j', 'risk_score', 'score_text', 'mortality_score', 'morbidity_score', 'other_reason', 'refer_other_text', 'over_8', '1-7_days', 'otd', 'proposed_lvl', 'actual_lvl', 'difference_nobed', 'difference_other', 'difference_other_text', 'difference_unknown']:
        print("The Patient CRF for site "+code+" has the wrong column index")
        intended = ['age', 'gender_male', 'gender_female', 'gender_non_binary', 'asian', 'black', 'mixed', 'white', 'other', 'cardiac', 'kidney', 'liver', 'neuro', 'pulm', 'dementia', 'diabetes', 'comp_diabetes', 'htn', 'malignancy', 'metastasis', 'pvd', 'rheum', 'surgical_speciality', 'proposed_procedure', 'cancer_surgery', 'ncepod_class', 'op_severity', 'scheduled_start', 'previous_cancellation', 'kts', 'cancelled', 'critical_care_7_days', 'encounter_start', 'encounter_end', 'los_encounter', 'los_0', 'los_1', 'los_2', 'los_3', 'died', 'died_date', 'cancel_nobed', 'cancel_other', 'cancel_other_text', 'sop', 'cpet', 'clin_j', 'risk_score', 'score_text', 'mortality_score', 'morbidity_score', 'other_reason', 'refer_other_text', 'over_8', '1-7_days', 'otd', 'proposed_lvl', 'actual_lvl', 'difference_nobed', 'difference_other', 'difference_other_text', 'difference_unknown']
        actual = list(patient.columns)
        print([item for item in actual if item not in intended])
        
    #loading time_series
    time_series = pd.read_excel("C:\\Users\\chris\\Documents\\Job Documents\\Portfolio Evidence\\Research\\Post-Operative Critical Care Beds Project\\Results Data\\Cleaned CRFs\\"+ts, sheet_name=None)
    try:
        time_series['Anonymised Data'] = time_series['Sheet1']
    except:
        pass
    try:
        time_series['Anonymised Data'] = time_series['Sheet 1']
    except:
        pass
    
    time_series = pd.DataFrame(data=time_series['Anonymised Data'])
    time_series.columns = time_series.iloc[0]
    time_series = time_series.iloc[1:94, 0:31]
    
    time_series['date'] = pd.to_datetime(time_series['date'], errors='coerce')
    time_series = time_series.set_index('date')
    time_series.index.name = None
    
    time_series = time_series.applymap(listify)
    
    if list(time_series.columns) != ['total_op', 'total_day', 'total_elec', 'total_emer', 'c_otd_cancel', 'c_otd_cancel_nobed', 'c_ref_lvl1', 'c_plan_adm_lvl1', 'c_adm_lvl1', 'c_ref_lvl2', 'c_plan_adm_lvl2', 'c_adm_lvl2', 'c_adm_7d_lvl2', 'c_ref_lvl3', 'c_plan_adm_lvl3', 'c_adm_lvl3', 'c_adm_7d_lvl3', 'otd_cancel', 'otd_cancel_nobed', 'ref_lvl1', 'plan_adm_lvl1', 'adm_lvl1', 'ref_lvl2', 'plan_adm_lvl2', 'adm_lvl2', 'adm_7d_lvl2', 'ref_lvl3', 'plan_adm_lvl3', 'adm_lvl3', 'adm_7d_lvl3']:
        print("The time series CRF for site "+code+" has the wrong column index")
        intended = ['total_op', 'total_day', 'total_elec', 'total_emer', 'c_otd_cancel', 'c_otd_cancel_nobed', 'c_ref_lvl1', 'c_plan_adm_lvl1', 'c_adm_lvl1', 'c_ref_lvl2', 'c_plan_adm_lvl2', 'c_adm_lvl2', 'c_adm_7d_lvl2', 'c_ref_lvl3', 'c_plan_adm_lvl3', 'c_adm_lvl3', 'c_adm_7d_lvl3', 'otd_cancel', 'otd_cancel_nobed', 'ref_lvl1', 'plan_adm_lvl1', 'adm_lvl1', 'ref_lvl2', 'plan_adm_lvl2', 'adm_lvl2', 'adm_7d_lvl2', 'ref_lvl3', 'plan_adm_lvl3', 'adm_lvl3', 'adm_7d_lvl3']
        actual = list(time_series.columns)
        print([item for item in actual if item not in intended])
        
    #updating critical care within 7 days column using time series and patient CRFs
    patient['level0-1_crit_care'] = crit_care(patient)
    patient['level0-1_crit_care_7d'] = crit_care_7d(patient, time_series)
    
    return [code, patient, time_series]

In [879]:
#function for deriving variables for the main analysis from the patient CRF
def pat_values(dataset):
    row_data = pd.DataFrame()
    
    #value extraction
    level0_1 = len(dataset[(dataset['actual_lvl'] < 2) & (dataset['cancelled'] == 0)])
    level0_1_crit_care = len(dataset[dataset['level0-1_crit_care'] == 1])
    level0_1_crit_care_7d = len(dataset[dataset['level0-1_crit_care_7d'] == 1])
    no_cancellations = dataset[dataset['cancelled'] == 0]
    no_cancellations_lvl_1 = dataset[(dataset['cancelled'] == 0) & (dataset['los_1'] > 0)]
    no_cancellations_lvl_2 = dataset[(dataset['cancelled'] == 0) & (dataset['los_2'] > 0)]
    no_cancellations_lvl_3 = dataset[(dataset['cancelled'] == 0) & (dataset['los_3'] > 0)]
    died = dataset[dataset['died'] == 1]
    died_6m = died[(died['died_date'] - died['kts']) < pd.Timedelta(182.5, unit='d')]
    risk_score = dataset[dataset['risk_score'] == 1]
    mortality = risk_score[risk_score['mortality_score'] > 0]
    morbidity = risk_score[risk_score['morbidity_score'] > 0]
    level_1_ref = dataset[dataset['proposed_lvl'] == 1]
    level_2_ref = dataset[dataset['proposed_lvl'] == 2]
    level_3_ref = dataset[dataset['proposed_lvl'] == 3]
    stepdowns_lvl1 = no_cancellations[(no_cancellations['proposed_lvl'] == 1) & (no_cancellations['actual_lvl'] == 0)]
    stepdowns_level23_to_0 = no_cancellations[(no_cancellations['proposed_lvl'] > 1) & (no_cancellations['actual_lvl'] == 0)]
    stepdowns_level23_to_1 = no_cancellations[(no_cancellations['proposed_lvl'] > 1) & (no_cancellations['actual_lvl'] == 1)]
    
    
    #variable creation
    row_data['avg_age'] = [dataset['age'].mean()]
    row_data['%female'] = dataset['gender_female'].mean()
    row_data['%white'] = dataset['white'].mean()
    row_data['%BAME'] = 1 - dataset['white'].mean()
    row_data['%asian'] = dataset['asian'].mean()
    row_data['%black'] = dataset['black'].mean()
    row_data['CCI'] = cci(dataset.mean(numeric_only=True))
    row_data['%cancer'] = dataset['cancer_surgery'].mean()
    row_data['avg_ncepod'] = dataset['ncepod_class'].mean()
    row_data['avg_severity'] = dataset['op_severity'].mean()
    row_data['%cancelled'] = dataset['cancelled'].mean()
    row_data['%cancelled_nobed'] = dataset['cancel_nobed'].mean()
    row_data['%level0-1_crit_care'] = level0_1_crit_care / level0_1
    row_data['%level0-1_crit_care_7d'] = level0_1_crit_care_7d / level0_1
    row_data['LOS_mean'] = no_cancellations['los_encounter'].mean()
    row_data['LOS_median'] = no_cancellations['los_encounter'].median()
    row_data['LOS1_mean'] = no_cancellations_lvl_1['los_1'].mean()
    row_data['LOS1_median'] = no_cancellations_lvl_1['los_1'].median()
    row_data['LOS2_mean'] = no_cancellations_lvl_2['los_2'].mean()
    row_data['LOS2_median'] = no_cancellations_lvl_2['los_2'].median()
    row_data['LOS3_mean'] = no_cancellations_lvl_3['los_3'].mean()
    row_data['LOS3_median'] = no_cancellations_lvl_3['los_3'].median()
    row_data['%died'] = no_cancellations['died'].mean()
    row_data['%died6m'] = len(died_6m) / len(no_cancellations)
    row_data['%clin_j'] = dataset['clin_j'].mean()
    row_data['%cpet'] = dataset['cpet'].mean()
    row_data['%risk_score'] = dataset['risk_score'].mean()
    row_data['pred_mortality'] = mortality['mortality_score'].mean()
    row_data['pred_morbidity'] = morbidity['morbidity_score'].mean()
    row_data['%sop'] = dataset['sop'].mean()
    row_data['%ref_over8'] = dataset['over_8'].mean()
    row_data['%ref_1_7'] = dataset['1-7_days'].mean()
    row_data['%ref_otd'] = dataset['otd'].mean()
    row_data['%_1_ref_over8'] = level_1_ref['over_8'].mean()
    row_data['%_1_ref_1_7'] = level_1_ref['1-7_days'].mean()
    row_data['%_1_ref_otd'] = level_1_ref['otd'].mean()
    row_data['%_2_ref_over8'] = level_2_ref['over_8'].mean()
    row_data['%_2_ref_1_7'] = level_2_ref['1-7_days'].mean()
    row_data['%_2_ref_otd'] = level_2_ref['otd'].mean()
    row_data['%_3_ref_over8'] = level_3_ref['over_8'].mean()
    row_data['%_3_ref_1_7'] = level_3_ref['1-7_days'].mean()
    row_data['%_3_ref_otd'] = level_3_ref['otd'].mean()
    row_data['%1stepdowns'] = len(stepdowns_lvl1) / len(no_cancellations[no_cancellations['proposed_lvl'] == 1])
    row_data['%1stepdowns_nobed'] = len(stepdowns_lvl1[stepdowns_lvl1['difference_nobed'] > 1]) / len(no_cancellations[no_cancellations['proposed_lvl'] == 1])
    row_data['%23to0stepdowns'] = len(stepdowns_level23_to_0) / len(no_cancellations[no_cancellations['proposed_lvl'] > 1])
    row_data['%23to0stepdowns_nobed'] = len(stepdowns_level23_to_0[stepdowns_level23_to_0['difference_nobed'] > 1]) / len(no_cancellations[no_cancellations['proposed_lvl'] > 1])
    row_data['%23to1stepdowns'] = len(stepdowns_level23_to_1) / len(no_cancellations[no_cancellations['proposed_lvl'] > 1])
    row_data['%23to1stepdowns_nobed'] = len(stepdowns_level23_to_1[stepdowns_level23_to_1['difference_nobed'] > 1]) / len(no_cancellations[no_cancellations['proposed_lvl'] > 1])

    return row_data

In [202]:
#function for deriving variables for the main analysis from the time series
def ts_values(dataset):
    row_data = pd.DataFrame()
    
    #value extraction
    total_op = dataset.iloc[91, 0]
    total_op = float(total_op[0])
    total_day = dataset.iloc[91, 1]
    total_day = float(total_day[0])
    surg_adm = total_op - total_day
    surg_adm_1000 = surg_adm / 1000
    total_elec = dataset.iloc[91, 2]
    total_elec = float(total_elec[0])
    total_emer = dataset.iloc[91, 3]
    total_emer = float(total_emer[0])
    otd_cancel = dataset.iloc[91, 4]
    otd_cancel = float(otd_cancel[0])
    
    otd_cancel_nobed = len(dataset['otd_cancel_nobed'].explode().dropna())
    ref_lvl1 = len(dataset['ref_lvl1'].explode().dropna())
    plan_adm_lvl1 = len(dataset['plan_adm_lvl1'].explode().dropna())
    adm_lvl1 = len(dataset['adm_lvl1'].explode().dropna())
    ref_lvl2 = len(dataset['ref_lvl2'].explode().dropna())
    plan_adm_lvl2 = len(dataset['plan_adm_lvl2'].explode().dropna())
    adm_lvl2 = len(dataset['adm_lvl2'].explode().dropna())
    adm_7d_lvl2 = len(dataset['adm_7d_lvl2'].explode().dropna())
    ref_lvl3 = len(dataset['ref_lvl3'].explode().dropna())
    plan_adm_lvl3 = len(dataset['plan_adm_lvl3'].explode().dropna())
    adm_lvl3 = len(dataset['adm_lvl3'].explode().dropna())
    adm_7d_lvl3 = len(dataset['adm_7d_lvl3'].explode().dropna())
    
    #variable creation
    row_data['total_op'] = [total_op]
    row_data['%day'] = total_day / total_op
    row_data['surg_adm'] = surg_adm
    row_data['%emerg'] = total_emer / (total_emer + total_elec)
    
    row_data['cancel_per_1000_total'] = otd_cancel / (total_op / 1000)
    row_data['cancel_nobed_per_1000'] = otd_cancel_nobed / surg_adm_1000
    
    row_data['ref_lvl1_per_1000'] = ref_lvl1 / surg_adm_1000
    row_data['plan_adm_lvl1_per_1000'] = plan_adm_lvl1 / surg_adm_1000
    row_data['adm_lvl1_per_1000'] = adm_lvl1 / surg_adm_1000
    row_data['%emerg_lvl1'] = (adm_lvl1 - plan_adm_lvl1) / adm_lvl1
    
    row_data['ref_lvl2_per_1000'] = ref_lvl2 / surg_adm_1000
    row_data['plan_adm_lvl2_per_1000'] = plan_adm_lvl2 / surg_adm_1000
    row_data['adm_lvl2_per_1000'] = adm_lvl2 / surg_adm_1000
    row_data['%emerg_lvl2'] = (adm_lvl2 - plan_adm_lvl2) / adm_lvl2
    
    row_data['ref_lvl3_per_1000'] = ref_lvl3 / surg_adm_1000
    row_data['plan_adm_lvl3_per_1000'] = plan_adm_lvl3 / surg_adm_1000
    row_data['adm_lvl3_per_1000'] = adm_lvl3 / surg_adm_1000
    row_data['%emerg_lvl3'] = (adm_lvl3 - plan_adm_lvl3) / adm_lvl3
    
    row_data['ref_lvl23_per_1000'] = (ref_lvl2 + ref_lvl3) / surg_adm_1000
    row_data['plan_adm_lvl23_per_1000'] = (plan_adm_lvl2 + plan_adm_lvl3) / surg_adm_1000
    row_data['adm_lvl23_per_1000'] = (adm_lvl2 + adm_lvl3) / surg_adm_1000
    row_data['%emerg_lvl23'] = ((adm_lvl2 + adm_lvl3) - (plan_adm_lvl2 + plan_adm_lvl3)) / (adm_lvl2 + adm_lvl3)
    
    return row_data

In [ ]:
#function to create a dataframe of all of the patients followed up with relevant information
def patients(dataset):
    

## Main analysis

In [691]:
# load survey dataframe
main = pd.read_excel("C:\\Users\\chris\\Documents\\Job Documents\\Portfolio Evidence\\Research\\Post-Operative Critical Care Beds Project\\Results Data\\survey_dataframe.xlsx", sheet_name=None)
main = pd.DataFrame(data=main['Sheet1'])
main = main.set_index('hospital_code')
main.index.name = None

#drop all without hospital code
main = main[main.index.notnull()]

display(main)

,hospital_name,type,icb,level_1,num_level_1,level_2,level_3,em_dept,services,mdt,risk_score,procedure_based,clinical_judgement,ref_anaes,ref_surg,ref_nurse,ref_spr,no_vetting,vet_anaes,vet_icu,vet_surg,vet_nurse,alo_anaes,alo_icu,alo_surg,alo_nurse,bed_nurse,nurse_bed_esc,cover_cons,cover_spr,cover_sho,clin_anaes,clin_icu,clin_surg,discharge_anaes,discharge_icu,discharge_surg,periph_vaso,central_vaso,cpap,bipap,stay_limit,all_spec,single_spec,level_1_ring,level_1_cap,level_1_esc,level_2_ring,level_2_cap,level_2_esc,level_3_ring,level_3_cap,level_3_esc,level_23_ring,level_23_cap,level_23_esc,totalringcap
RWDDA,Lincoln County Hospital,2,NHS Lincolnshire ICB,1,1,0,1,1,"5, 11, 13, 15",0,1,0,1,1,0,0,0,0,1,0,0,0,1,0,0,0,2.67,3.67,1,0,0,0,0,1,0,0,1,0,0,0,0,0,1.0,0.0,0,8,10,0,0,0,0,11,15,0,11,15,0
RXR20,Royal Blackburn Teaching Hospital,1,NHS Lancashire and South Cumbria ICB,1,1,1,1,1,"8, 11, 13, 16",0,1,1,0,1,1,0,0,0,1,1,0,0,1,1,1,0,3.00,4.00,0,1,0,0,1,1,0,1,1,1,1,1,0,0,1.0,0.0,0,4,4,0,6,6,0,18,22,0,24,28,0
RMC01,Royal Bolton Hospital,2,NHS Greater Manchester ICB,0,0,1,1,1,"13, 15",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,10,10,0,8,8,0,18,18,0
R0A02,Manchester Royal Infirmary,1,NHS Greater Manchester ICB,1,1,1,1,1,"2, 5, 8, 10, 11, 13, 14, 15, 16",0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,2.00,3.00,0,1,0,1,1,1,1,0,0,1,1,1,0,0,1.0,0.0,9,9,9,0,20,20,0,28,28,0,48,48,9
RADP3,North Middlesex Hospital,2,NHS North Central London ICB,0,0,1,1,1,13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,11,11,0,12,12,0,23,23,0
RJ1AK,William Harvey Hospital,2,NHS Kent and Medway ICB,0,0,0,1,1,"6, 10, 11, 13,15",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,18,18,0,18,18,0
RQM01,Chelsea and Westminster Hospital,1,NHS North West London ICB,1,1,0,1,1,"1, 3, 5, 10, 11, 13",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1.00,1.00,0,1,0,1,0,1,1,0,1,0,0,1,0,1,1.0,0.0,0,0,4,0,0,0,0,14,22,0,14,22,0
RFRPA,Rotherham Hospital,2,NHS South Yorkshire ICB,0,0,1,1,1,"5, 11, 13",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,8,8,0,5,5,0,13,13,0
7A1A4,Wrexham Maelor Hospital,2,Betsi Cadwalader University HB,1,1,0,1,1,"5, 10, 11, 13, 15, 16",0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1.50,2.00,1,0,0,1,1,1,0,0,1,1,0,1,1,1,1.0,0.0,0,3,3,0,0,0,0,12,12,0,12,12,0
RJC02,Warwick Hospital,2,NHS Coventry and Warwickshire ICB,0,0,1,1,1,"5, 13",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.00,0.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,2,2,0,5,5,0,7,7,0


In [282]:
#iterate through CRF directory
p_crf = []
ts_crf = []

for file in os.listdir("C:\\Users\\chris\\Documents\\Job Documents\\Portfolio Evidence\\Research\\Post-Operative Critical Care Beds Project\\Results Data\\Cleaned CRFs"):
    if "_p.xlsx" in file:
        p_crf.append(str(file))
    if "_ts.xlsx" in file:
        ts_crf.append(file)
        
#create list of pairs to iterate over and derive file paths and check for unpaired datasets
for index, file in enumerate(p_crf):
    truncated = file.replace("_p.xlsx", "")
    if truncated+"_ts.xlsx" in ts_crf:
        p_crf[index] = [file, truncated+"_ts.xlsx"]
        ts_crf.remove(truncated+"_ts.xlsx")

paired = [item for item in p_crf if len(item) == 2]
p_crf = [item for item in p_crf if item not in paired]

print("The following files have no pairs")
print(p_crf)
print(ts_crf)

The following files have no pairs
[]
[]


In [380]:
#derive values from each pair in directory and add to dataframes
for pair in paired:
    hosp_data = data_load(pair[0], pair[1])

In [880]:
#test cell
hosp_data = data_load(paired[0][0], paired[0][1])
display(pat_values(hosp_data[1]))
display(hosp_data[1])
#display(ts_values(hosp_data[2]))
#display(hosp_data[2])

,avg_age,%female,%white,%BAME,%asian,%black,CCI,%cancer,avg_ncepod,avg_severity,%cancelled,%cancelled_nobed,%level0-1_crit_care,%level0-1_crit_care_7d,LOS_mean,LOS_median,LOS1_mean,LOS1_median,LOS2_mean,LOS2_median,LOS3_mean,LOS3_median,%died,%died6m,%clin_j,%cpet,%risk_score,pred_mortality,pred_morbidity,%sop,%ref_over8,%ref_1_7,%ref_otd,%_1_ref_over8,%_1_ref_1_7,%_1_ref_otd,%_2_ref_over8,%_2_ref_1_7,%_2_ref_otd,%_3_ref_over8,%_3_ref_1_7,%_3_ref_otd,%1stepdowns,%1stepdowns_nobed,%23to0stepdowns,%23to0stepdowns_nobed,%23to1stepdowns,%23to1stepdowns_nobed
0,65.328859,0.42953,0.791946,0.208054,0.073826,0.006711,3.550336,0.630872,3.852349,4.409396,0.053691,0.0,0.025974,0.025974,13.375887,8.0,3.436782,3.0,3.872881,3.0,3.277778,2.0,0.078014,0.078014,0.409396,0.33557,0.42953,0.038409,0.204138,0.006711,0.510067,0.167785,0.322148,0.561644,0.123288,0.315068,0.470588,0.220588,0.308824,0.375,0.125,0.5,0.0,0.0,0.0,0.0,0.148649,0.0


,age,gender_male,gender_female,gender_non_binary,asian,black,mixed,white,other,cardiac,kidney,liver,neuro,pulm,dementia,diabetes,comp_diabetes,htn,malignancy,metastasis,pvd,rheum,surgical_speciality,proposed_procedure,cancer_surgery,ncepod_class,op_severity,scheduled_start,previous_cancellation,kts,cancelled,critical_care_7_days,encounter_start,encounter_end,los_encounter,los_0,los_1,los_2,los_3,died,died_date,cancel_nobed,cancel_other,cancel_other_text,sop,cpet,clin_j,risk_score,score_text,mortality_score,morbidity_score,other_reason,refer_other_text,over_8,1-7_days,otd,proposed_lvl,actual_lvl,difference_nobed,difference_other,difference_other_text,difference_unknown,level0-1_crit_care,level0-1_crit_care_7d
RXR2029,67,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,Head and Neck,"Tracheostomy, bilateral neck dissection, left ...",1,4,3,2023-09-04 08:58:00,0,2023-09-04 10:22:00,0,1,2023-09-03 12:20:00,2023-10-14 15:50:00,41,30,0,0.0,11.0,0,0,0,0,0,0,0,0,1,0,0.0183,0.0,0,0,1,0,0,3,3,0,0,0,0,Not applicable,Not applicable
RXR20400,51,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,ENT,"Tracheostomy, microlaryngoscopy and biopsy",0,3,2,2023-10-16 10:36:00,0,2023-10-16 10:50:00,0,1,2023-10-13 11:59:00,2023-11-30 17:49:00,48,47,0,0.0,1.0,0,0,0,0,0,0,0,1,0,0,0.0000,0.0,0,0,0,0,1,3,3,0,0,0,0,Not applicable,Not applicable
RXR20331,56,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,Maxillofacial,tracheostomy and neck disection,1,4,5,2023-10-09 08:54:00,0,2023-10-09 10:17:00,0,1,2023-10-09 07:05:00,2023-10-25 11:17:00,16,13,0,1.0,2.0,0,0,0,0,0,0,0,0,0,0,0.0000,0.0,0,0,1,0,0,3,3,0,0,0,0,Not applicable,Not applicable
RXR2098,69,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,0,Maxillofacial,Excision of failed flap,1,3,5,2023-09-11 00:00:00,0,2023-09-11 10:48:00,0,1,2023-09-02 09:45:00,2023-12-04 22:30:00,93,89,0,0.0,4.0,0,0,0,0,0,0,0,1,0,0,0.0000,0.0,1,Tracheostomy,0,0,1,3,3,0,0,0,0,Not applicable,Not applicable
RXR20557,46,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,Hepatobiliary,Pancreatectomy,0,4,5,2023-11-06 00:00:00,0,2023-11-06 11:45:00,0,1,2023-11-05 23:04:00,2023-11-14 19:13:00,9,6,0,0.0,3.0,0,0,0,0,0,0,0,0,1,Sortv2,0.0089,0.0,0,0,1,0,0,3,3,0,0,0,0,Not applicable,Not applicable
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
RXR20777,83,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,Vascular,EVAR,0,4,5,2023-11-30 00:00:00,0,2023-11-30 10:22:00,0,1,2023-11-30 08:14:00,2023-12-01 18:54:00,1,0,1,0.0,0.0,0,0,0,0,0,0,1,0,1,0,0.0248,0.2,0,0,1,0,0,1,1,0,0,0,0,0,0
RXR20778,69,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,1,0,Vascular,Axillo-Bifemoral Bypass,0,4,5,2023-11-30 00:00:00,0,2023-11-30 11:17:00,0,1,2023-11-23 16:24:00,2023-12-30 13:38:00,37,32,5,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0.0000,0.0,0,0,0,0,1,1,1,0,0,0,0,0,0
RXR20400,51,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,ENT,"Total laryngectomy, neck dissection and total ...",1,4,5,2023-11-13 00:00:00,0,2023-11-13 11:14:00,0,1,2023-10-16 00:00:00,2023-11-30 17:47:00,45,41,4,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0.0000,0.0,0,0,0,0,1,1,1,0,0,0,0,0,0
RXR20502,29,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,General surgery,Laparoscopic right hemicolectomy,0,4,5,2023-10-27 00:00:00,0,2023-10-27 14:31:00,0,1,2023-10-27 20:55:00,2023-10-31 17:36:00,4,0,4,0.0,0.0,0,0,0,0,0,0,0,1,0,0,0.0000,0.0,0,0,1,0,0,1,1,0,0,0,0,0,0


In [ ]:
#Data transformation
#Sum LOS in each area of care as total LOS
#Ignore clinical judgement field as this is almost always checked
#Calculate total number of stepdowns from data without cancellations as code will automatically add zeros where there is empty data

#Analysis
#Code to summarise the key statistics from time series and patient CRF and save them in a dataframe where the index is hospital
#Code to summarise patient characteristics by saving all patients into 1 dataframe, add hospital details for each patient
#Look at characteristics of the patients referred to level 1 versus other levels
#Analyse the impact of particular structures of enhanced care (e.g. nurse referral) on what type of patients get admitted, referral to capacity ratio etc
#Penalisation via LASSO for analysis
#Look into fragility index for conclusions
#Calculate referral to resource ratio
#Patient dataset should look at the characteristics of those referred to level 1 vs level 2/3 ?lower CCI/acuity
#Quantify the number of patients that were referred to level 1 that were cancelled due to a lack of bed, compare to level 2/3 ?were any level 1 patients cancelled due to a lack of bed
#Compare cancellations due to a lack of bed between the time-series and patient CRFs and assess concordance